# Linear Regression Code Along

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('LinReg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for row in data.head(1)[0]:
    print(row)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'], outputCol='features')


In [14]:
output = assembler.transform(data)

In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [21]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.682104978287384|
| -4.645221115133211|
| 10.377574472675974|
|  6.365394329461196|
|-21.470799549014487|
| -8.926334213992504|
| 21.691095803318376|
| 2.9933791864924615|
| -5.737742174616699|
| -6.306232745086277|
|-10.660230959491798|
|-15.117664041652915|
| -3.043037067479304|
|  17.45866113316339|
|-27.257029771986822|
| -9.420812317528885|
|-1.5711258345551755|
|-6.1356320705423855|
| -6.278681625019601|
|-11.667196201869501|
+-------------------+
only showing top 20 rows



In [25]:
test_results.rootMeanSquaredError

10.137053157466218

In [26]:
test_results.r2

0.9844400638608444

In [27]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
unlabled_data = test_data.select('features')

In [29]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.7656188210424...|
|[31.8093003166791...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = lr_model.transform(unlabled_data)

In [31]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|  331.610974781481|
|[30.4925366965402...|287.11646683504773|
|[30.7377203726281...| 451.4031677235539|
|[30.9716756438877...| 488.2732154274315|
|[31.1239743499119...|508.41785338878026|
|[31.1280900496166...| 566.1790209610472|
|[31.2834474760581...| 570.0899936223491|
|[31.3091926408918...|429.72733865344117|
|[31.5171218025062...| 281.6561628250024|
|[31.5257524169682...|450.27185955496816|
|[31.5261978982398...| 419.7547571518296|
|[31.5741380228732...| 559.5269362022398|
|[31.5761319713222...| 544.2696210568076|
|[31.6098395733896...| 427.0868885179448|
|[31.6739155032749...|  502.982097681868|
|[31.7207699002873...| 548.1957457955518|
|[31.7216523605090...| 349.3480524664278|
|[31.7242025238451...| 509.5235193585029|
|[31.7656188210424...|502.83276326062673|
|[31.8093003166791...| 548.4390955647107|
+--------------------+------------